In [ ]:
# Image denoiser
# Copyright Andriy Melnykov 2025

# Images are TIFF, 16-bit RGB

# --- 1. paths (edit!) ---------------------------------------------------------
MODEL_PATH = "/content/drive/MyDrive/Colab_Data/train_16.keras"
IN_TIF     = "/content/drive/MyDrive/Colab_Data/test_image.tiff"
OUT_TIF    = "/content/drive/MyDrive/Colab_Data/test_image_filtered_16.tiff"

# --- 2. drive & libs ----------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

!pip install tifffile  # fast 16‑bit IO
!pip install imagecodecs

import numpy as np
import tifffile as tiff
import math
import tqdm
import tensorflow as tf
from tensorflow import keras

print("Libraries loaded")

# --- 3. load model (no compile needed for inference) --------------------------
model = keras.models.load_model(MODEL_PATH, compile=False)
PATCH_IN   = 128
PATCH_KEEP = 64            # central crop returned to mosaic
STRIDE     = 32

# Hann window for soft overlap‑add (same size as KEEP region)
win1d = np.hanning(PATCH_KEEP)
window = np.outer(win1d, win1d).astype('float32') + 1e-6  # avoid /0 later
window = window[..., None]                                # add channel dim

# --- 4. read & normalise full frame ------------------------------------------
img16 = tiff.imread(IN_TIF)                 # (H,W,3) uint16
img   = img16.astype('float32') / 65535.0   # 0‑1

print("Image loaded")

H, W, C = img.shape
pad_tile = (PATCH_IN - PATCH_KEEP) // 2          # 32 px
pad_img = PATCH_IN
pad_cfg_tile = ((pad_tile, pad_tile), (pad_tile, pad_tile), (0,0))
pad_cfg_img = ((pad_img, pad_img), (pad_img, pad_img), (0,0))
img_padded = np.pad(img, pad_cfg_img, mode='reflect')
Hpad, Wpad = img_padded.shape[:2]

# allocate output & weight accumulators
out  = np.zeros_like(img_padded, dtype='float32')
wmap = np.zeros_like(img_padded, dtype='float32')

# tile ranges
ys = range(0, Hpad - PATCH_IN + 1, STRIDE)
xs = range(0, Wpad - PATCH_IN + 1, STRIDE)

# --- 5. process tiles in small batches ---------------------------------------
batch = []
coords = []
BATCH_SIZE = 32

def flush_batch():
    if not batch: return
    pred = model.predict(np.stack(batch), verbose=0)
    for (y,x), p in zip(coords, pred):
        # central crop
        p_crop = p[pad_tile:pad_tile+PATCH_KEEP, pad_tile:pad_tile+PATCH_KEEP] * window
        oy, ox = y+pad_tile, x+pad_tile
        out[oy:oy+PATCH_KEEP, ox:ox+PATCH_KEEP] += p_crop
        wmap[oy:oy+PATCH_KEEP, ox:ox+PATCH_KEEP] += window
    batch.clear(); coords.clear()

for y in tqdm.tqdm(ys, desc="Tiles"):
    for x in xs:
        patch = img_padded[y:y+PATCH_IN, x:x+PATCH_IN]
        batch.append(patch)
        coords.append((y,x))
        if len(batch) == BATCH_SIZE:
            flush_batch()
flush_batch()   # leftovers

# --- 6. normalise by weight map & crop back to original size -----------------
wmap[wmap == 0] = 1e-6
out /= wmap
out_clipped = np.clip(out, 0.0, 1.0)
denoised = out_clipped[pad_img:pad_img+H, pad_img:pad_img+W]

# --- 7. save 16‑bit TIFF ------------------------------------------------------
tiff.imwrite(OUT_TIF, (denoised*65535).astype('uint16'))
print("Image saved:", OUT_TIF)